In [1]:
import torch
torch.cuda.empty_cache()

import time
import json
from transformers import AutoTokenizer, AutoModelForCausalLM
from pydantic import BaseModel, Field, RootModel
from typing import Optional, Union, Literal, ForwardRef, List, Any
from enum import Enum
from guidance import models, system, user, assistant, json as gen_json, gen
import torch, outlines
from transformers import AutoModelForCausalLM, AutoTokenizer
import json
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from pydantic import BaseModel, Field, RootModel
from typing import Optional, Union
from enum import Enum
from guidance import models, system, user, assistant, json as gen_json
import guidance
from utils import timing_decorator

In [2]:

with open("../output/2025-airport-charges-terms-and-conditions/tinychargesmarkdown.md", "r") as f:
    markdown_content = f.read()


In [ ]:

MODEL_ID = "Qwen/Qwen3-30B-A3B"

hf_model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    device_map="auto",               
    low_cpu_mem_usage=True,          
)


tok        = AutoTokenizer.from_pretrained(MODEL_ID)


Loading checkpoint shards:   0%|          | 0/16 [00:00<?, ?it/s]

In [ ]:

model = guidance.models.Transformers(hf_model, tok)

In [ ]:
from pydantic import BaseModel, Field
from typing import List, Any

class DomainVariable(BaseModel):
    """Defines a single variable the LLM can use in the computation graph."""
    name: str = Field(..., description="The unique identifier for the variable.")
    description: str = Field(..., description="A detailed explanation of what this variable represents.")
    # Optional: You could add type hints, units, etc. for more advanced validation
    unit: Optional[str] = Field(..., description="The unit of the variable")
    data_type : type = Field(..., description="The data type of the variable")


In [ ]:
ALL_VARIABLES = {
    
    'transfer_passenger_count': DomainVariable(name='transfer_passenger_count', description='Total number of transferring passengers.', unit=None, data_type=float),
    'airline_scheduling_season': DomainVariable(name='airline_scheduling_season', description='Whether summer/winter airline scheduling season.', unit=None, data_type=float),
    'takeoff_aircraft_mtow_tonnes': DomainVariable(name='takeoff_aircraft_mtow_tonnes', description='The Maximum Take-Off Weight in tonnes.', unit='tonne', data_type=float),
    'landing_aircraft_mtow_tonnes': DomainVariable(name='landing_aircraft_mtow_tonnes', description='The Maximum Landing Weight in tonnes.', unit='tonne', data_type=float),
    
    'parking_duration_hours': DomainVariable(
        name='parking_duration_hours',
        description='Total duration of parking in hours. Surcharges apply at 48 and 72 hours.',
        unit='hours',
        data_type=float
    ),
    'aircraft_stand_type': DomainVariable(
        name='aircraft_stand_type',
        description='The type of aircraft stand used for parking. E.g., "Wide Contact", "Narrow Remote", "LAP", "Long Term Remote".',
        unit=None,
        data_type=str
    ),
    'parking_location': DomainVariable(
        name='parking_location',
        description='The location of the parking stand, either "EAP" (East Aerodrome Parking) or "WAP" (West Aerodrome Parking).',
        unit=None,
        data_type=str
    ),
    'is_overnight_parking': DomainVariable(
        name='is_overnight_parking',
        description='True if the parking occurs during the free overnight period (2300-0600hrs).',
        unit=None,
        data_type=bool
    )
}
# 2. Map charge types to the variable names they are allowed to use
CHARGE_CATEGORY_VARIABLES = {
    # --- Existing Categories ---
    "transfer_passenger_charge": [
        'transfer_passenger_count', 
        'airline_scheduling_season'
    ],
    "runway_landing_charge": [
        'landing_aircraft_mtow_tonnes', 
        'airline_scheduling_season'
    ],
    "runway_takeoff_charge": [
        'takeoff_aircraft_mtow_tonnes', 
        'airline_scheduling_season'
    ],

    # --- New Categories for Parking Charges ---
    "east_aerodrome_parking_charge": [
        'parking_duration_hours',
        'aircraft_stand_type',
        'is_overnight_parking'
        # 'parking_location' is implicitly 'EAP' for this category
    ],
    "west_aerodrome_parking_charge": [
        'parking_duration_hours',
        'aircraft_stand_type',
        'is_overnight_parking'
        # 'parking_location' is implicitly 'WAP' for this category
    ]
}



In [ ]:
def create_dynamic_variable_enum(charge_category: str) -> type(Enum):
    """
    Creates a new Enum class containing only the variables relevant
    to the specified charge category.
    """
    variable_names = CHARGE_CATEGORY_VARIABLES.get(charge_category)
    if not variable_names:
        raise ValueError(f"Unknown charge category: {charge_category}")
    
    # The dictionary for the Enum must have {MEMBER_NAME: value}
    # We'll use uppercase for the member name for convention.
    enum_dict = {name.upper(): name for name in variable_names}
    
    # Create the Enum class dynamically
    return Enum("Var", enum_dict)

Var = create_dynamic_variable_enum("transfer_passenger_charge")
print(Var.TRANSFER_PASSENGER_COUNT.value)

In [ ]:
allowed_variables = [el.value for el in list(Var)]
allowed_variables

In [ ]:
from pydantic import BaseModel, Field, RootModel
from typing import Union, Literal
from enum import Enum

# Separate enums for clarity and type safety
class MathOperator(str, Enum):
    ADD = "ADD"
    MULTIPLY = "MULTIPLY"
    DIVIDE = "DIVIDE"

class Comparator(str, Enum):
    GREATER_THAN = "GREATER_THAN"
    LESS_THAN = "LESS_THAN"
    EQUAL_TO = "EQUAL_TO"

class Units(str, Enum):
    HOURS = "HOURS"
    MINUTES = "MINUTES"
    EUROS = "EUROS"
    UNITLESS = "UNITLESS"
    
# --- Node Definitions ---

class ValueNode(BaseModel):
    type: Literal["VALUE"] = "VALUE"
    value: float
    description: str
    unit: Units

class VariableNode(BaseModel):
    type: Literal["VARIABLE"] = "VARIABLE"
    name: str 
    description: str
    unit: Units

class BinaryOpNode(BaseModel):
    """Node for mathematical operations that produce a number."""
    type: Literal["BINARY_OPERATION"] = "BINARY_OPERATION"
    operator: MathOperator
    left: 'AnyNode'
    right: 'AnyNode'

class ComparisonNode(BaseModel):
    """Node for comparison operations that produce a boolean."""
    type: Literal["COMPARISON"] = "COMPARISON"
    operator: Comparator
    left: 'AnyNode'
    right: 'AnyNode'

class ConditionalNode(BaseModel):
    """Node for if-then-else logic."""
    type: Literal["CONDITIONAL"] = "CONDITIONAL"
    condition: ComparisonNode # Condition must be a comparison
    if_true: 'AnyNode'
    if_false: 'AnyNode'

# --- Recursive Setup ---

AnyNode = Union[
    ValueNode, 
    VariableNode, 
    BinaryOpNode, 
    ConditionalNode
]

# Use model_rebuild() to safely resolve all forward references
BinaryOpNode.model_rebuild()
ConditionalNode.model_rebuild()
ComparisonNode.model_rebuild()

class Node(RootModel):
    root: BinaryOpNode

In [16]:
@guidance
def create_graph_with_cot(llm, allowed_variables_prompt, document, query, output_schema):
    
    with system():
        llm += f"""You are an expert system that converts textual calculation rules into structured JSON expression trees.
        You MUST think step-by-step and reason before generating the final JSON.

        **Reasoning Guidelines:**
        1.  **Identify All Values From Document Required for Charge Calculation:** Find the value's that are required for computing the charge present in the user query.
        2.  **Identify Variables:** List the symbolic variables needed for the calculation (e.g., parking_duration_hours) from the list below.
        **Allowed Variables for this Task:**
        ---
        {allowed_variables_prompt}
        ---
        
        3.  **Synthesize Plan:** Briefly describe how you will combine these pieces into a final expression tree.
        4. **Rethink and Finalize Approach**: Before processing with generation, rethink your progress so far and make adjustments if necessary, then finalize and proceed to generate the expression tree.

        Your reasoning starts when you encounter <think> token. After writing your reasoning, you MUST output `</think>` on a new line and then you WILL generate the JSON object.

        
        """

    with user():
        llm += f"""
        **Document:**
        ---
        {document}
        ---

        **Query:**
        Based on the document, construct the computation graph for the following request:
        "{query}"
        
        """

    with assistant():
        llm += "<think>"
        llm += "I will now follow the reasoning guidelines step-by-step before generating the final JSON.\n"
        llm += gen("thought", stop="</think>", max_tokens=1000)

        # After thinking, it generates the JSON.
        llm += "\n\nFinal JSON object:\n"
        llm += gen_json(
            name="result_graph", 
            schema=output_schema,
            max_tokens=1000 
        )
        
    return llm

In [17]:

class ComputationGraphBuilder:
    """
    Orchestrates the creation of a computation graph by preparing dynamic
    constraints and prompting the LLM.
    """
    
    def __init__(self, model):
        """
        Initializes the builder with a guidance model.
        """
        self.model = model
        # Set the default LLM for all guidance programs
        # guidance.llm = self.model
    @timing_decorator
    def build(self, document_content: str, query: str, charge_category: str) -> dict:
        """
        Generates a computation graph for a given query and document.

        Args:
            document_content: The text containing the rules.
            query: A natural language question about what to calculate.
            charge_category: The specific charge context used to filter variables.

        Returns:
            A dictionary representing the computation graph or an error.
        """
        print(f"--- Building graph for charge category: '{charge_category}' ---")
        
        # 1. Dynamically create the filtered Enum for this specific task
        try:
            Var = create_dynamic_variable_enum(charge_category)
        except ValueError as e:
            print(f"Error: {e}")
            return {"error": str(e)}

        # 3. Create a formatted prompt string of allowed variables for the LLM
        allowed_variables = [el.value for el in list(Var)]
        allowed_variables_prompt = "\n".join(
            [f"- **{v.name}**: {v.description}" for name, v in ALL_VARIABLES.items() if name in allowed_variables]
        )

        try:
            # 4. Execute the guidance program with all dynamic components
            result_lm = self.model + create_graph_with_cot(
                allowed_variables_prompt=allowed_variables_prompt,
                document=document_content,
                query=query,
                output_schema=Node
            )
            
            
            # print("\nSuccessfully generated graph:")
            # # Use model_dump_json for Pydantic v2
            # print(pydantic_graph.model_dump_json(indent=2)) 
            return result_lm
            
        except Exception as e:
            print(f"\nAn error occurred while building the graph for '{query}': {e}")
            return {"error": str(e)}

In [23]:
graph_builder = ComputationGraphBuilder(model=model)

my_query = "Calculate the total parking charge for a Long Term Remote stand type at West Aerodrome Parking with no overnight parking. The parking duration is a variable with unit hours."
my_charge_category = "west_aerodrome_parking_charge"

# 3. Call the build method to generate the graph
start_time = time.perf_counter()
expression_tree_dict = graph_builder.build(
    document_content=markdown_content,
    query=my_query,
    charge_category=my_charge_category
)
end_time = time.perf_counter()
execution_time = end_time - start_time
# print(f"Execution time for '{func.__name__}': {execution_time:.4f} seconds")

--- Building graph for charge category: 'west_aerodrome_parking_charge' ---


StitchWidget(initial_height='auto', initial_width='100%', srcdoc='<!doctype html>\n<html lang="en">\n<head>\n …

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [24]:
execution_time
# # expression_tree_dict['thought']
# loaded_json = json.loads(expression_tree_dict['result_graph'])
# print(json.dumps(loaded_json, indent=4))

288.8231407201383

In [27]:
# execution_time
print(expression_tree_dict['thought'])
# loaded_json = json.loads(expression_tree_dict['result_graph'])
# print(json.dumps(loaded_json, indent=4))


Okay, let's tackle this problem. The user wants to calculate the total parking charge for a Long Term Remote stand at West Aerodrome Parking (WAP), with no overnight parking and a variable parking duration in hours. 

First, I need to identify all the values required from the document. The main factors here are the stand type (Long Term Remote), the parking duration, and whether there's overnight parking. The document mentions that overnight parking from 2300-0600 is free, but the user specified no overnight parking, so that part might not apply here. 

Looking at the West Aerodrome Parking table, the standard charge for Long Term Remote is 180.00 euros per day or part thereof. Wait, the charging basis is per 15 minutes except for Long Term Remote, which is per day. But the user's parking duration is given in hours. So I need to convert the duration into days. However, the problem states that the duration is a variable in hours, so maybe the calculation needs to handle that.

Next, th

In [26]:
# execution_time
# expression_tree_dict['thought']
loaded_json = json.loads(expression_tree_dict['result_graph'])
print(json.dumps(loaded_json, indent=4))

{
    "type": "BINARY_OPERATION",
    "operator": "ADD",
    "left": {
        "type": "CONDITIONAL",
        "condition": {
            "type": "COMPARISON",
            "operator": "GREATER_THAN",
            "left": {
                "type": "VARIABLE",
                "name": "parking_duration_hours",
                "description": "Total duration of parking in hours. Surcharges apply at 48 and 72 hours.",
                "unit": "HOURS"
            },
            "right": {
                "type": "VALUE",
                "value": 72,
                "description": "Threshold for surcharge application at 72 hours.",
                "unit": "HOURS"
            }
        },
        "if_true": {
            "type": "BINARY_OPERATION",
            "operator": "MULTIPLY",
            "left": {
                "type": "VALUE",
                "value": 3,
                "description": "Surcharge multiplier for parking over 72 hours (200% additional).",
                "unit": "MINUTES"


In [36]:
graph_builder = ComputationGraphBuilder(model=model)

my_query = "Calculate the total parking charge for a Long Term Remote stand type at West Aerodrome Parking with no overnight parking. The parking duration is a variable with unit hours."
my_charge_category = "west_aerodrome_parking_charge"

# 3. Call the build method to generate the graph
start_time = time.perf_counter()
expression_tree_dict = graph_builder.build(
    document_content=markdown_content,
    query=my_query,
    charge_category=my_charge_category
)
end_time = time.perf_counter()
execution_time = end_time - start_time
# print(f"Execution time for '{func.__name__}': {execution_time:.4f} seconds")

--- Building graph for charge category: 'west_aerodrome_parking_charge' ---


StitchWidget(initial_height='auto', initial_width='100%', srcdoc='<!doctype html>\n<html lang="en">\n<head>\n …

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [37]:
execution_time
# print(expression_tree_dict['thought'])
# loaded_json = json.loads(expression_tree_dict['result_graph'])
# print(json.dumps(loaded_json, indent=4))

303.79992694291286

In [38]:
# execution_time
print(expression_tree_dict['thought'])
# loaded_json = json.loads(expression_tree_dict['result_graph'])
# print(json.dumps(loaded_json, indent=4))


1. **Identify All Values From Document Required for Charge Calculation:**

The user is asking for the total parking charge for a Long Term Remote stand type at West Aerodrome Parking (WAP) with no overnight parking and a variable parking duration in hours.

From the document, the relevant information is:

- **Standard Charge per Aircraft/Stand type for Long Term Remote at WAP:** €180.00 per day or part thereof (from the WAP table).
- **Parking surcharges for WAP:**
  - 48 hours up to 72 hours (including night-time): Standard rate + 0% (i.e., standard rate)
  - 72 hours and over (including night-time): Standard rate + 200%
- **Overnight parking (2300-0600hrs) is free of charge.** However, the user specified "no overnight parking," so this surcharge does not apply.
- **The parking duration is a variable with unit hours.** The calculation must account for the duration in hours and convert it into days or part thereof for the standard rate, then apply surcharges based on duration.

2. **I

In [39]:
# execution_time
# print(expression_tree_dict['thought'])
loaded_json = json.loads(expression_tree_dict['result_graph'])
print(json.dumps(loaded_json, indent=4))

{
    "type": "BINARY_OPERATION",
    "operator": "ADD",
    "left": {
        "type": "BINARY_OPERATION",
        "operator": "MULTIPLY",
        "left": {
            "type": "BINARY_OPERATION",
            "operator": "DIVIDE",
            "left": {
                "type": "VARIABLE",
                "name": "parking_duration_hours",
                "description": "Total duration of parking in hours.",
                "unit": "HOURS"
            },
            "right": {
                "type": "VALUE",
                "value": 24,
                "description": "Number of hours in a day.",
                "unit": "HOURS"
            }
        },
        "right": {
            "type": "VALUE",
            "value": 1,
            "description": "Rounding up to the next whole day.",
            "unit": "HOURS"
        }
    },
    "right": {
        "type": "VALUE",
        "value": 180.0,
        "description": "Standard charge per day for Long Term Remote at West Aerodrome Parking."

In [42]:
graph_builder = ComputationGraphBuilder(model=model)

my_query = "Calculate the total parking charge for a Long Term Remote stand type at West Aerodrome Parking with no overnight parking. The parking duration is a variable with unit hours."
my_charge_category = "west_aerodrome_parking_charge"

# 3. Call the build method to generate the graph
start_time = time.perf_counter()
expression_tree_dict = graph_builder.build(
    document_content=markdown_content,
    query=my_query,
    charge_category=my_charge_category
)
end_time = time.perf_counter()
execution_time = end_time - start_time
# print(f"Execution time for '{func.__name__}': {execution_time:.4f} seconds")

--- Building graph for charge category: 'west_aerodrome_parking_charge' ---


StitchWidget(initial_height='auto', initial_width='100%', srcdoc='<!doctype html>\n<html lang="en">\n<head>\n …

In [43]:
execution_time
# print(expression_tree_dict['thought'])
# loaded_json = json.loads(expression_tree_dict['result_graph'])
# print(json.dumps(loaded_json, indent=4))

334.2381346898619

In [44]:
# execution_time
print(expression_tree_dict['thought'])
# loaded_json = json.loads(expression_tree_dict['result_graph'])
# print(json.dumps(loaded_json, indent=4))


1. **Identify All Values From Document Required for Charge Calculation:**

The user is asking for the total parking charge for a Long Term Remote stand type at West Aerodrome Parking (WAP) with no overnight parking and a variable parking duration in hours.

From the document, the relevant information is:

- **Standard charge per aircraft/stand type for Long Term Remote at WAP**: €180.00 per day or part thereof.
- **Parking surcharges for WAP**:
  - 48 hours up to 72 hours (including night-time): Standard rate + 0% (i.e., standard rate)
  - 72 hours and over (including night-time): Standard rate + 200%
- **Overnight parking (2300-0600hrs) is free of charge**, but the user specified "no overnight parking", so this surcharge does not apply.
- **Parking duration is a variable with unit hours**.

2. **Identify Variables:**

The variables needed for the calculation are:

- **parking_duration_hours**: The total duration of parking in hours.
- **aircraft_stand_type**: "Long Term Remote" (spec

In [45]:
# execution_time
# print(expression_tree_dict['thought'])
loaded_json = json.loads(expression_tree_dict['result_graph'])
print(json.dumps(loaded_json, indent=4))

{
    "type": "BINARY_OPERATION",
    "operator": "ADD",
    "left": {
        "type": "CONDITIONAL",
        "condition": {
            "type": "COMPARISON",
            "operator": "GREATER_THAN",
            "left": {
                "type": "VARIABLE",
                "name": "parking_duration_hours",
                "description": "Total duration of parking in hours. Surcharges apply at 48 and 72 hours.",
                "unit": "HOURS"
            },
            "right": {
                "type": "VALUE",
                "value": 0,
                "description": "Minimum parking duration",
                "unit": "HOURS"
            }
        },
        "if_true": {
            "type": "BINARY_OPERATION",
            "operator": "MULTIPLY",
            "left": {
                "type": "VALUE",
                "value": 3,
                "description": "Surcharge multiplier for parking duration over 72 hours",
                "unit": "MINUTES"
            },
            "right":

In [12]:
graph_builder = ComputationGraphBuilder(model=model)

my_query = "Calculate the total parking charge for a Long Term Remote stand type at West Aerodrome Parking with no overnight parking. The parking duration is a variable with unit hours."
my_charge_category = "west_aerodrome_parking_charge"

# 3. Call the build method to generate the graph
start_time = time.perf_counter()
expression_tree_dict = graph_builder.build(
    document_content=markdown_content,
    query=my_query,
    charge_category=my_charge_category
)
end_time = time.perf_counter()
execution_time = end_time - start_time
# print(f"Execution time for '{func.__name__}': {execution_time:.4f} seconds")

--- Building graph for charge category: 'west_aerodrome_parking_charge' ---


StitchWidget(initial_height='auto', initial_width='100%', srcdoc='<!doctype html>\n<html lang="en">\n<head>\n …

In [13]:
execution_time
# print(expression_tree_dict['thought'])
# loaded_json = json.loads(expression_tree_dict['result_graph'])
# print(json.dumps(loaded_json, indent=4))

287.3310044519603

In [14]:
# execution_time
print(expression_tree_dict['thought'])
# loaded_json = json.loads(expression_tree_dict['result_graph'])
# print(json.dumps(loaded_json, indent=4))


1. **Identify All Values From Document Required for Charge Calculation:**
   - The parking stand type is "Long Term Remote" at West Aerodrome Parking (WAP).
   - The parking duration is a variable in hours.
   - The parking duration is not during overnight hours (2300-0600hrs), so no free overnight charge applies.
   - The standard charge for Long Term Remote at WAP is €180.00 per day or part thereof.
   - Surcharges apply for parking durations of 48-72 hours (standard rate +100%) and 72+ hours (standard rate +200%).

2. **Identify Variables:**
   - **parking_duration_hours**: The total duration of parking in hours.
   - **aircraft_stand_type**: "Long Term Remote" (though this is fixed in this case, it's still a variable in the formula).
   - **is_overnight_parking**: False (since the query specifies no overnight parking).

3. **Synthesize Plan:**
   - The base charge is €180.00 per day or part thereof. Since the duration is in hours, we need to convert it to days (e.g., 25 hours = 2 

In [15]:
# execution_time
# print(expression_tree_dict['thought'])
loaded_json = json.loads(expression_tree_dict['result_graph'])
print(json.dumps(loaded_json, indent=4))

{
    "type": "BINARY_OPERATION",
    "operator": "ADD",
    "left": {
        "type": "CONDITIONAL",
        "condition": {
            "type": "COMPARISON",
            "operator": "GREATER_THAN",
            "left": {
                "type": "VARIABLE",
                "name": "parking_duration_hours",
                "description": "Total duration of parking in hours. Surcharges apply at 48 and 72 hours.",
                "unit": "HOURS"
            },
            "right": {
                "type": "VALUE",
                "value": 72,
                "description": "Threshold for 200% surcharge.",
                "unit": "HOURS"
            }
        },
        "if_true": {
            "type": "BINARY_OPERATION",
            "operator": "MULTIPLY",
            "left": {
                "type": "VALUE",
                "value": 180.0,
                "description": "Standard charge for Long Term Remote at WAP.",
                "unit": "EUROS"
            },
            "right": {


In [18]:
graph_builder = ComputationGraphBuilder(model=model)

my_query = "Calculate the total parking charge for a Long Term Remote stand type at West Aerodrome Parking with no overnight parking. The parking duration is a variable with unit hours."
my_charge_category = "west_aerodrome_parking_charge"

# 3. Call the build method to generate the graph
start_time = time.perf_counter()
expression_tree_dict = graph_builder.build(
    document_content=markdown_content,
    query=my_query,
    charge_category=my_charge_category
)
end_time = time.perf_counter()
execution_time = end_time - start_time
# print(f"Execution time for '{func.__name__}': {execution_time:.4f} seconds")

--- Building graph for charge category: 'west_aerodrome_parking_charge' ---


StitchWidget(initial_height='auto', initial_width='100%', srcdoc='<!doctype html>\n<html lang="en">\n<head>\n …

In [19]:
execution_time
# print(expression_tree_dict['thought'])
# loaded_json = json.loads(expression_tree_dict['result_graph'])
# print(json.dumps(loaded_json, indent=4))

411.2895414577797

In [20]:
# execution_time
print(expression_tree_dict['thought'])
# loaded_json = json.loads(expression_tree_dict['result_graph'])
# print(json.dumps(loaded_json, indent=4))


1. **Identify All Values From Document Required for Charge Calculation:**
   - The parking stand type is "Long Term Remote" at West Aerodrome Parking (WAP).
   - The parking duration is a variable in hours.
   - The parking duration is not during overnight hours (2300-0600hrs), so no free overnight charge applies.
   - The standard charge for Long Term Remote at WAP is €180.00 per day or part thereof.
   - Surcharges apply for parking durations of 48-72 hours (standard rate +100%) and 72+ hours (standard rate +200%).

2. **Identify Variables:**
   - **parking_duration_hours**: The total duration of parking in hours.
   - **aircraft_stand_type**: "Long Term Remote" (though this is fixed in this case, it's still a variable in the formula).
   - **is_overnight_parking**: False (since the query specifies no overnight parking).

3. **Synthesize Plan:**
   - The base charge is €180.00 per day or part thereof. Since the duration is in hours, we need to convert it to days (e.g., 25 hours = 2 

In [21]:
# execution_time
# print(expression_tree_dict['thought'])
loaded_json = json.loads(expression_tree_dict['result_graph'])
print(json.dumps(loaded_json, indent=4))

{
    "type": "BINARY_OPERATION",
    "operator": "ADD",
    "left": {
        "type": "BINARY_OPERATION",
        "operator": "MULTIPLY",
        "left": {
            "type": "BINARY_OPERATION",
            "operator": "DIVIDE",
            "left": {
                "type": "VARIABLE",
                "name": "parking_duration_hours",
                "description": "Total duration of parking in hours. Surcharges apply at 48 and 72 hours.",
                "unit": "HOURS"
            },
            "right": {
                "type": "VALUE",
                "value": 24,
                "description": "Hours in a day",
                "unit": "HOURS"
            },
            "description": "Convert parking duration from hours to days"
        },
        "right": {
            "type": "VALUE",
            "value": 1,
            "description": "Ceiling function to round up to the next day",
            "unit": "UNITLESS"
        },
        "description": "Calculate the number of days 